## Final Project Submission

Please fill out:
* Student name: Minna Fingerhood
* Student pace: full time
* Scheduled project review date/time: 
* Instructor name: Fangfang
* Blog post URL:


In [ ]:
# Your code here - remember to use markdown cells for comments as well!

In [ ]:
#how does date affect the price of houses?
#heatmap of lat/long at house prices?
#how does bathroom and square footage -- high correlation demonstrated by heatmap 

assingment: 
    - You'll clean, explore, and model this dataset with a multivariate linear regression to predict the sale price of houses as accurately as possible. 

questions to keep in mind:

    -"how did you pick the question(s) that you did?"
    -"why are these questions important from a business perspective?"
    -"how did you decide on the data cleaning options you performed?"
    -"why did you choose a given method or library?"
    -"why did you select those visualizations and what did you learn from each of them?"
    -"why did you pick those features as predictors?"
    -"how would you interpret the results?"
    -"how confident are you in the predictive quality of the results?"
    -"what are some of the things that could cause the results to be wrong?"

In [2]:
#import data and packages
import pandas as pd
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt 
import statsmodels.formula.api as smf
#import functions as f
%matplotlib inline


In [4]:
df = pd.read_csv('kc_house_data.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [18]:
def log_transform(df, cols):
    ''' takes in dataframe and cols to log transfrom and returns 
        the dataset with the log transformed columns dropped the regular
        col'''
    for col in cols:
        df[col] = np.log(df[col])
        df['log_{}'.format(col)] = df[col]
        df.drop(col, axis=1, inplace=True)
    return df
        
        

In [19]:
df_log = log_transform(df,['price', 'sqft_above'])
df_log.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,log_price,log_sqft_above
0,7129300520,10/13/2014,3,1.00,1180,5650,1.0,NaN,0.0,3,...,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650,12.309982,7.073270
1,6414100192,12/9/2014,3,2.25,2570,7242,2.0,0.0,0.0,3,...,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639,13.195614,7.682482
2,5631500400,2/25/2015,2,1.00,770,10000,1.0,0.0,0.0,3,...,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062,12.100712,6.646391
3,2487200875,12/9/2014,4,3.00,1960,5000,1.0,0.0,0.0,5,...,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000,13.311329,6.956545
4,1954400510,2/18/2015,3,2.00,1680,8080,1.0,0.0,0.0,3,...,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503,13.142166,7.426549


In [ ]:
#data cleaning code: 

#duplicates are a result of the same house being sold another time... Therefore do not need to delete 
df[df.id.duplicated(keep=False)].head()

#yr_renovated column has both nan and 0.0 filler values... changes all to nan so 0.0 doesn't skew the data 
df['yr_renovated'] = df['yr_renovated'].replace(0.0, np.nan)


#Should replace null values with 0 because categorical for the column.
#going to try with converting null values to 0.0
df['waterfront'] = df['waterfront'].fillna(0.0)

#had to apply log function to columns(price, sqft_above, sqft_living)

#sqft_basement was in string format, so had to convert and then replaced 0.0 with null to find median without skewed outliers
#used median to replace null values so that it did not alter mean too much... mean went from 740 to 716 once we replaced 
df['sqft_basement'] = pd.to_numeric(df['sqft_basement'], errors='coerce')
df['sqft_basement'] = df['sqft_basement'].replace(0.0, np.nan)
df['sqft_basement'] = df['sqft_basement'].fillna(df['sqft_basement'].median())

## made graphs determining the frequency of each variable as well as heat map for correlation (looked specifically at price row)
##need to figure out which ones to use in model by determining which are most evenly distributed and most correlated to price 
df.corr()[abs(df.corr() > 0.75)] #determining high correlations 

#found that bathrooms, seqft_living, grade, sqft_above, sqft_living15 are most correlated to price 





In [ ]:
df.head()

In [ ]:
df.view.value_counts()

In [ ]:
df.view.value_counts()

In [ ]:
#Inspecting Data: Waterfront and yr_renovated have a lot of null objects... 
df.info()

In [ ]:
df.describe()

In [ ]:
#check & inspect duplicates
df[df.id.duplicated(keep=False)].head()
#duplicates are a result of the same house being sold another time... Therefore do not need to delete 

In [ ]:
#Cleaning Data!! 

In [ ]:
#yr_renovated column has both nan and 0.0 filler values... changes all to nan so 0.0 doesn't skew the data 
df['yr_renovated'] = df['yr_renovated'].replace(0.0, np.nan)

In [ ]:
df['yr_renovated'].describe()

In [ ]:
#df.waterfront.value_counts() #for waterfronts we have 19075 entries for 0.0
#df.waterfront.isnull().value_counts() #we also have 2376 null values... Should replace null values with 0 because categorical for the column. But are null values the same as no waterfront or was the information just unavailable?
#df[df.waterfront.isnull()].head()
#going to try with converting null values to 0.0
df['waterfront'] = df['waterfront'].fillna(0.0)


In [ ]:
df.waterfront.isnull().value_counts()

In [ ]:
df.columns

In [ ]:
df['sqft_basement'] = pd.to_numeric(df['sqft_basement'], errors='coerce')
df['sqft_basement'] = df['sqft_basement'].replace(0.0, np.nan)
df['sqft_basement'] = df['sqft_basement'].fillna(df['sqft_basement'].median())

In [ ]:
df['sqft_basement'].describe()

In [ ]:
#clear outliers
def clean_outliers(df):
    for col in df.columns:
        q1 = df['{}'.format(col)].quantile(0.25)
        q3 = df['{}'.format(col)].quantile(0.75)
        iqr = q3-q1 #Interquartile range
        fence_low  = q1-1.5*iqr
        fence_high = q3+1.5*iqr
        df_out = df.loc[(df['{}'.format(col)] > fence_low) & (df['{}'.format(col)] < fence_high)]
        return df_out

In [ ]:
clean_df  = clean_outliers(df)

In [ ]:
clean_df.sqft_above.transform(np.log)

In [ ]:
clean_df.hist(bins=50, figsize=(20,15))

In [ ]:
df.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
sns.heatmap(abs(df.corr()), cmap = 'BuPu', linewidth = .5)


In [ ]:
sns.scatterplot(x = df.yr_renovated, y=np.log(df.price))


# df.corr()[(df.corr() > 0.5)]

In [ ]:
sqft_vs_others = smf.ols(formula = 'sqft_living~sqft_above+sqft_basement+sqft_living15+sqft_lot15',data=df).fit()
sqft_vs_others.summary()

In [ ]:

model = smf.ols(formula='price ~ bathrooms + sqft_living + grade + sqft_above + sqft_living15', data = df).fit()
model.summary()

In [5]:
from datetime import datetime

df['date'] =  pd.to_datetime(df['date'], format='%m/%d/%Y')
df['month'] = df['date'].dt.month #made new column for months 

df['season'] = df.month.apply(lambda dt: (df.month%12 + 3)//3)


print(df.month)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/minnafingerhood/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-56ea1629d7a9>", line 6, in <module>
    df['season'] = df.month.apply(lambda dt: (df.month%12 + 3)//3)
  File "/Users/minnafingerhood/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py", line 3370, in __setitem__
    self._set_item(key, value)
  File "/Users/minnafingerhood/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py", line 3445, in _set_item
    value = self._sanitize_column(key, value)
  File "/Users/minnafingerhood/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py", line 3619, in _sanitize_column
    value = reindexer(value).T
  File "/Users/minnafingerhood/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py", line 3589, in reindexer
    value = value._values.copy()
KeyboardInterrupt

During handling of the 

KeyboardInterrupt: 

In [6]:
from datetime import datetime

df['date'] =  pd.to_datetime(df['date'], format='%m/%d/%Y')
df['month'] = df['date'].dt.month #made new column for months 

df['season'] = df.month.apply(lambda month: (df.month%12 + 3)//3)


print(df.season)

KeyboardInterrupt: 